In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from tqdm import tqdm

from stance_utils import *
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 23
np.random.seed(RANDOM_SEED)

torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda', index=0)

In [3]:
classes = {'FAVOR': np.array([1, 0, 0]), 'AGAINST': np.array([0, 1, 0]), 'NONE': np.array([0, 0, 1])}
classes_ = np.array(['FAVOR', 'AGAINST', 'NONE'])

In [4]:
def train_and_test(train_file, test_file, target):
    
    sentence_maxlen = 0
    x_train = []
    y_train = []

    
    with open(train_file, 'r') as trainfile:
        for line in trainfile:
            
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            
            #if line[0].strip() != 'ID' and line[1].strip() == t:
            if line[0].strip() != 'ID' and target in line[1].strip():
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_train.append(tweet)
                y_train.append(classes[line[3].strip()])
                
#     with open('rich_data_feminism_athesim_abortion.txt','r') as newfile:
#         for line in newfile:
#             line = line.replace('#SemST', '').strip()
#             line = line.split('\t')
#             tweet = line[0]
#             tweet = process_tweet(tweet)
#             if len(tweet) > sentence_maxlen:
#                 sentence_maxlen = len(tweet)
#             x_train.append(tweet)
#             y_train.append(classes[line[1].strip()])

    pp = len(x_train)
    x_test = []
    y_test = []
    with open(test_file, 'r') as testfile:
        for line in testfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        

            #if line[0] != 'ID' and line[1] == t:
            if line[0] != 'ID' and target in line[1].strip():
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_test.append(tweet)
                y_test.append(classes[line[3].strip()])


    
    return x_train, y_train, x_test, y_test, sentence_maxlen,pp





In [5]:
train_data_file_m = '/data/parush/stance_mohammed/train.txt'
test_data_file_m = '/data/parush/stance_mohammed/test.txt'
TARGETS_m = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]


# train_data_file_s = '/data/parush/SomasundaranWiebe-politicalDebates/train.txt'
# test_data_file_s = '/data/parush/SomasundaranWiebe-politicalDebates/test.txt'
# TARGETS_s = ['god','healthcare','guns','gayRights','abortion', 'creation']


# train_data_file_q = '/data/parush/Data_MPCHI/train.txt'
# test_data_file_q = '/data/parush/Data_MPCHI/test.txt'
# TARGETS_q = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#       'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#       'Should women take HRT post-menopause?']


In [6]:
class CustomDataset(Dataset):
    
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.title = dataframe['TITLE']
        self.targets = self.data.target_list
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [7]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 3)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids,return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

In [8]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)



In [9]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

In [10]:
import shutil, sys   
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [11]:
def train_model(start_epochs,  n_epochs, valid_loss_min_input, 
                training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
    valid_loss_min = valid_loss_min_input 
    for epoch in range(start_epochs, n_epochs+1):
        train_loss = 0
        valid_loss = 0

        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            #print('yyy epoch', batch_idx)
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            #if batch_idx%5000==0:
             #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print('before loss data in training', loss.item(), train_loss)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
            #print('after loss data in training', loss.item(), train_loss)

        print('############# Epoch {}: Training End     #############'.format(epoch))

        print('############# Epoch {}: Validation Start   #############'.format(epoch))
        ######################    
        # validate the model #
        ######################

        model.eval()
        val_targets, val_outputs = [] , []
        with torch.no_grad():
            for batch_idx, data in enumerate(validation_loader, 0):
                ids = data['ids'].to(device, dtype = torch.long)
                mask = data['mask'].to(device, dtype = torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
                targets = data['targets'].to(device, dtype = torch.float)
                outputs = model(ids, mask, token_type_ids)

                loss = loss_fn(outputs, targets)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))                
                
                val_targets.extend(targets.cpu().detach().numpy().tolist())
                val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            print('############# Epoch {}: Validation End     #############'.format(epoch))
            # calculate average losses
            #print('before cal avg train loss', train_loss)
            train_loss = train_loss/len(training_loader)
            valid_loss = valid_loss/len(validation_loader)
            # print training/validation statistics 
            print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
                epoch, 
                train_loss,
                valid_loss
                ))
            # create checkpoint variable and add important data
            checkpoint = {
                'epoch': epoch + 1,
                'valid_loss_min': valid_loss,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }

            # save checkpoint
            save_ckp(checkpoint, False, checkpoint_path, best_model_path)
            ## TODO: save the model if validation loss has decreased
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
                # save checkpoint as best model
                save_ckp(checkpoint, True, checkpoint_path, best_model_path)
                valid_loss_min = valid_loss

        print('############# Epoch {}  Done   #############\n'.format(epoch))
    
    return model,val_targets, val_outputs

In [12]:
# In[273]:
for target in TARGETS_m:
    
    MAX_LEN = 128
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 32
    EPOCHS = 20
    LEARNING_RATE = 1e-05
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    train_texts, train_labels, test_texts, test_labels, sen_maxlen, pp = train_and_test(train_data_file_m, test_data_file_m, target)
    df2 = pd.DataFrame([])
    df2['TITLE'] = train_texts
    df2['target_list'] = train_labels
    df_test = pd.DataFrame([])
    df_test['TITLE'] = test_texts
    df_test['target_list'] = test_labels
    train_size = 0.8
    train_dataset = df2.sample(frac=train_size,random_state=200)
    valid_dataset = df2.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)


    print("FULL Dataset: {}".format(df2.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(valid_dataset.shape))
    
    training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
    validation_set = CustomDataset(valid_dataset,  tokenizer, MAX_LEN)
    train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

    test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

    training_loader = DataLoader(training_set, **train_params)
    validation_loader = DataLoader(validation_set, **test_params)
    model = BERTClass()
    model.to(device)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
    checkpoint_path = '/home/p/parush/bert_data/current_checkpoint_abortion.pt'
    best_model = '/home/p/parush/bert_data/best_model_trained_on_abortion.pt'
    trained_model,val_targets, val_outputs = train_model(1, EPOCHS, np.Inf, training_loader, validation_loader, model, 
                      optimizer,checkpoint_path,best_model)
    val_preds = (np.array(val_outputs) > 0.5).astype(int)
    print('Trained and tested on: ', target)
    print(classification_report(val_targets, val_preds,labels=[0,1],digits = 4))




FULL Dataset: (513, 2)
TRAIN Dataset: (410, 2)
TEST Dataset: (103, 2)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [8]:
# df2 = pd.DataFrame([])
# df2['TITLE'] = train_texts
# df2['target_list'] = train_labels

In [9]:
# df_test = pd.DataFrame([])
# df_test['TITLE'] = test_texts
# df_test['target_list'] = test_labels

In [1]:
# df_test

In [2]:
# df2

In [12]:
# MAX_LEN = 128
# TRAIN_BATCH_SIZE = 32
# VALID_BATCH_SIZE = 32
# EPOCHS = 20
# LEARNING_RATE = 1e-05
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [13]:
# class CustomDataset(Dataset):
    
#     def __init__(self, dataframe, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.data = dataframe
#         self.title = dataframe['TITLE']
#         self.targets = self.data.target_list
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.title)

#     def __getitem__(self, index):
#         title = str(self.title[index])
#         title = " ".join(title.split())

#         inputs = self.tokenizer.encode_plus(
#             title,
#             None,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             return_token_type_ids=True,
#             truncation=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']
#         token_type_ids = inputs["token_type_ids"]


#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
#             'targets': torch.tensor(self.targets[index], dtype=torch.float)
#         }

In [14]:
# train_size = 0.8
# train_dataset = df2.sample(frac=train_size,random_state=200)
# valid_dataset = df2.drop(train_dataset.index).reset_index(drop=True)
# train_dataset = train_dataset.reset_index(drop=True)


# print("FULL Dataset: {}".format(df2.shape))
# print("TRAIN Dataset: {}".format(train_dataset.shape))
# print("TEST Dataset: {}".format(valid_dataset.shape))

FULL Dataset: (653, 2)
TRAIN Dataset: (522, 2)
TEST Dataset: (131, 2)


In [15]:
# training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
# validation_set = CustomDataset(valid_dataset,  tokenizer, MAX_LEN)

In [16]:
# train_params = {'batch_size': TRAIN_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }

# test_params = {'batch_size': VALID_BATCH_SIZE,
#                 'shuffle': False,
#                 'num_workers': 0
#                 }

# training_loader = DataLoader(training_set, **train_params)
# validation_loader = DataLoader(validation_set, **test_params)

In [3]:
# class BERTClass(torch.nn.Module):
#     def __init__(self):
#         super(BERTClass, self).__init__()
#         self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
#         self.l2 = torch.nn.Dropout(0.3)
#         self.l3 = torch.nn.Linear(768, 3)
    
#     def forward(self, ids, mask, token_type_ids):
#         _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids,return_dict=False)
#         output_2 = self.l2(output_1)
#         output = self.l3(output_2)
#         return output

# model = BERTClass()
# model.to(device)

In [18]:
# def loss_fn(outputs, targets):
#     return torch.nn.BCEWithLogitsLoss()(outputs, targets)

# optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [19]:
# def load_ckp(checkpoint_fpath, model, optimizer):
#     """
#     checkpoint_path: path to save checkpoint
#     model: model that we want to load checkpoint parameters into       
#     optimizer: optimizer we defined in previous training
#     """
#     # load check point
#     checkpoint = torch.load(checkpoint_fpath)
#     # initialize state_dict from checkpoint to model
#     model.load_state_dict(checkpoint['state_dict'])
#     # initialize optimizer from checkpoint to optimizer
#     optimizer.load_state_dict(checkpoint['optimizer'])
#     # initialize valid_loss_min from checkpoint to valid_loss_min
#     valid_loss_min = checkpoint['valid_loss_min']
#     # return model, optimizer, epoch value, min validation loss 
#     return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

In [20]:
# import shutil, sys   
# def save_ckp(state, is_best, checkpoint_path, best_model_path):
#     """
#     state: checkpoint we want to save
#     is_best: is this the best checkpoint; min validation loss
#     checkpoint_path: path to save checkpoint
#     best_model_path: path to save best model
#     """
#     f_path = checkpoint_path
#     # save checkpoint data to the path given, checkpoint_path
#     torch.save(state, f_path)
#     # if it is a best model, min validation loss
#     if is_best:
#         best_fpath = best_model_path
#         # copy that checkpoint file to best path given, best_model_path
#         shutil.copyfile(f_path, best_fpath)

In [21]:
# def train_model(start_epochs,  n_epochs, valid_loss_min_input, 
#                 training_loader, validation_loader, model, 
#                 optimizer, checkpoint_path, best_model_path):
   
#   # initialize tracker for minimum validation loss
#     valid_loss_min = valid_loss_min_input 
#     for epoch in range(start_epochs, n_epochs+1):
#         train_loss = 0
#         valid_loss = 0

#         model.train()
#         print('############# Epoch {}: Training Start   #############'.format(epoch))
#         for batch_idx, data in enumerate(training_loader):
#             #print('yyy epoch', batch_idx)
#             ids = data['ids'].to(device, dtype = torch.long)
#             mask = data['mask'].to(device, dtype = torch.long)
#             token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#             targets = data['targets'].to(device, dtype = torch.float)

#             outputs = model(ids, mask, token_type_ids)

#             optimizer.zero_grad()
#             loss = loss_fn(outputs, targets)
#             #if batch_idx%5000==0:
#              #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             #print('before loss data in training', loss.item(), train_loss)
#             train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
#             #print('after loss data in training', loss.item(), train_loss)

#         print('############# Epoch {}: Training End     #############'.format(epoch))

#         print('############# Epoch {}: Validation Start   #############'.format(epoch))
#         ######################    
#         # validate the model #
#         ######################

#         model.eval()
#         val_targets, val_outputs = [] , []
#         with torch.no_grad():
#             for batch_idx, data in enumerate(validation_loader, 0):
#                 ids = data['ids'].to(device, dtype = torch.long)
#                 mask = data['mask'].to(device, dtype = torch.long)
#                 token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#                 targets = data['targets'].to(device, dtype = torch.float)
#                 outputs = model(ids, mask, token_type_ids)

#                 loss = loss_fn(outputs, targets)
#                 valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))                
                
#                 val_targets.extend(targets.cpu().detach().numpy().tolist())
#                 val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
#             print('############# Epoch {}: Validation End     #############'.format(epoch))
#             # calculate average losses
#             #print('before cal avg train loss', train_loss)
#             train_loss = train_loss/len(training_loader)
#             valid_loss = valid_loss/len(validation_loader)
#             # print training/validation statistics 
#             print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
#                 epoch, 
#                 train_loss,
#                 valid_loss
#                 ))
#             # create checkpoint variable and add important data
#             checkpoint = {
#                 'epoch': epoch + 1,
#                 'valid_loss_min': valid_loss,
#                 'state_dict': model.state_dict(),
#                 'optimizer': optimizer.state_dict()
#             }

#             # save checkpoint
#             save_ckp(checkpoint, False, checkpoint_path, best_model_path)
#             ## TODO: save the model if validation loss has decreased
#             if valid_loss <= valid_loss_min:
#                 print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
#                 # save checkpoint as best model
#                 save_ckp(checkpoint, True, checkpoint_path, best_model_path)
#                 valid_loss_min = valid_loss

#         print('############# Epoch {}  Done   #############\n'.format(epoch))
    
#     return model,val_targets, val_outputs

In [22]:
# checkpoint_path = '/home/p/parush/bert_data/current_checkpoint_abortion.pt'
# best_model = '/home/p/parush/bert_data/best_model_trained_on_abortion.pt'
# trained_model,val_targets, val_outputs = train_model(1, EPOCHS, np.Inf, training_loader, validation_loader, model, 
#                       optimizer,checkpoint_path,best_model)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.036592 	Average Validation Loss: 0.111198
Validation loss decreased (inf --> 0.111198).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.034328 	Average Validation Loss: 0.108167
Validation loss decreased (0.111198 --> 0.108167).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [24]:
# val_preds = (np.array(val_outputs) > 0.5).astype(int)

# print(classification_report(val_targets, val_preds,labels=[0,1],digits = 4))

              precision    recall  f1-score   support

           0     0.5667    0.6071    0.5862        28
           1     0.7468    0.8194    0.7815        72

   micro avg     0.6972    0.7600    0.7273       100
   macro avg     0.6568    0.7133    0.6838       100
weighted avg     0.6964    0.7600    0.7268       100
 samples avg     0.5725    0.5802    0.5751       100



/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
